In [1]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et

In [1]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et

xmlfiles=glob('./data_images/*.xml')
replace_text=lambda x:x.replace('\\','/')
xmlfiles=list(map(replace_text,xmlfiles))


tree=et.parse('./data_images/maksssksksss0.xml')
root=tree.getroot()

#Step2
#Read xml files,from each  xml files we need to extract filename,size(w,h),object name(xmin,xmax,ymin,ymax,name)
def extract_text(filename):
    tree=et.parse(filename)
    root=tree.getroot()
    #Extract filename
    image_name=root.find('filename').text
    #width and height of the img
    width=root.find('size').find('width').text
    height=root.find('size').find('height').text
    objs=root.findall('object')
    parser=[]
    for obj in objs:
        name=obj.find('name').text
        bndbox=obj.find('bndbox')
        xmin=bndbox.find('xmin').text
        xmax=bndbox.find('xmax').text
        ymin=bndbox.find('ymin').text
        ymax=bndbox.find('ymax').text
        parser.append([image_name,width,height,name,xmin,xmax,ymin,ymax])
        
    return parser




parser_all=list(map(extract_text,xmlfiles))

data=reduce(lambda x,y:x+y,parser_all)

df=pd.DataFrame(data,columns=['filename','width','height','name','xmin','xmax','ymin','ymax'])

cols=['width','height','xmin','xmax','ymin','ymax']
df[cols]=df[cols].astype(int)

#centerx,centery
df['center_x']=((df['xmax']+df['xmin'])/2)/df['width']
df['center_y']=((df['ymax']+df['ymin'])/2)/df['height']

#w
df['w']=(df['xmax']-df['xmin'])/df['width']
#height
df['h']=(df['ymax']-df['ymin'])/df['height']

#split the data into train and test
images=df['filename'].unique()
len(images)

img_df=pd.DataFrame(images,columns=['filename'])
img_train=tuple(img_df.sample(frac=0.8)['filename'])
img_test=tuple(img_df.query(f'filename not in{img_train}')['filename'])

train_df=df.query(f'filename in {img_train}')
test_df=df.query(f'filename in {img_test}')

#Assign id numbers to object names
def label_encoding(x):
    labels={'with_mask':0,'without_mask':1,'mask_weared_incorrect':2}
    return labels[x]

train_df['id']=train_df['name'].apply(label_encoding)
test_df['id']=test_df['name'].apply(label_encoding)


import os
from shutil import move
train_folder='data_images/train'
test_folder='data_images/test'

os.mkdir(train_folder)
os.mkdir(test_folder)

cols=['filename','id','center_x','center_y','w','h']
groupby_obj_train=train_df[cols].groupby('filename')
groupby_obj_test=test_df[cols].groupby('filename')

def save_data(filename,folder_path,group_obj):
    #move img
    src=os.path.join('D:\\yolo_v\\1_Data_preprocessing\\data_images',filename)
    dst=os.path.join(folder_path,filename)
    move(src,dst)
    #save the labels
    text_filename=os.path.join(folder_path,
                               os.path.splitext(filename)[0]+'.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep=' ',index=False,header=False)


filename_series =pd.Series(groupby_obj_train.groups.keys())
filename_series = filename_series.apply(lambda x: x.replace('\x00', '')) # Replace NULL bytes with an empty string
filename_series.apply(save_data,args=(train_folder,groupby_obj_train))

C:\Users\illas\AppData\Local\Temp\ipykernel_4544\14522858.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id']=train_df['name'].apply(label_encoding)
C:\Users\illas\AppData\Local\Temp\ipykernel_4544\14522858.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id']=test_df['name'].apply(label_encoding)


0      None
1      None
2      None
3      None
4      None
       ... 
677    None
678    None
679    None
680    None
681    None
Length: 682, dtype: object

In [3]:
tree=et.parse('./data_images/maksssksksss0.xml')
root=tree.getroot()

In [4]:
#Step2
#Read xml files,from each  xml files we need to extract filename,size(w,h),object name(xmin,xmax,ymin,ymax,name)
def extract_text(filename):
    tree=et.parse(filename)
    root=tree.getroot()
    #Extract filename
    image_name=root.find('filename').text
    #width and height of the img
    width=root.find('size').find('width').text
    height=root.find('size').find('height').text
    objs=root.findall('object')
    parser=[]
    for obj in objs:
        name=obj.find('name').text
        bndbox=obj.find('bndbox')
        xmin=bndbox.find('xmin').text
        xmax=bndbox.find('xmax').text
        ymin=bndbox.find('ymin').text
        ymax=bndbox.find('ymax').text
        parser.append([image_name,width,height,name,xmin,xmax,ymin,ymax])
        
    return parser



In [5]:
parser_all=list(map(extract_text,xmlfiles))

In [6]:
parser_all

[[['maksssksksss0.png',
   '512',
   '366',
   'without_mask',
   '79',
   '109',
   '105',
   '142'],
  ['maksssksksss0.png', '512', '366', 'with_mask', '185', '226', '100', '144'],
  ['maksssksksss0.png',
   '512',
   '366',
   'without_mask',
   '325',
   '360',
   '90',
   '141']],
 [['maksssksksss1.png', '400', '156', 'with_mask', '321', '354', '34', '69'],
  ['maksssksksss1.png', '400', '156', 'with_mask', '224', '261', '38', '73'],
  ['maksssksksss1.png', '400', '156', 'with_mask', '299', '315', '58', '81'],
  ['maksssksksss1.png', '400', '156', 'with_mask', '143', '174', '74', '115'],
  ['maksssksksss1.png', '400', '156', 'with_mask', '74', '95', '69', '99'],
  ['maksssksksss1.png', '400', '156', 'with_mask', '191', '221', '67', '93'],
  ['maksssksksss1.png', '400', '156', 'with_mask', '21', '44', '73', '93'],
  ['maksssksksss1.png', '400', '156', 'with_mask', '369', '398', '70', '99'],
  ['maksssksksss1.png',
   '400',
   '156',
   'without_mask',
   '83',
   '111',
   '56',
 

In [7]:
data=reduce(lambda x,y:x+y,parser_all)

In [8]:
data

[['maksssksksss0.png',
  '512',
  '366',
  'without_mask',
  '79',
  '109',
  '105',
  '142'],
 ['maksssksksss0.png', '512', '366', 'with_mask', '185', '226', '100', '144'],
 ['maksssksksss0.png',
  '512',
  '366',
  'without_mask',
  '325',
  '360',
  '90',
  '141'],
 ['maksssksksss1.png', '400', '156', 'with_mask', '321', '354', '34', '69'],
 ['maksssksksss1.png', '400', '156', 'with_mask', '224', '261', '38', '73'],
 ['maksssksksss1.png', '400', '156', 'with_mask', '299', '315', '58', '81'],
 ['maksssksksss1.png', '400', '156', 'with_mask', '143', '174', '74', '115'],
 ['maksssksksss1.png', '400', '156', 'with_mask', '74', '95', '69', '99'],
 ['maksssksksss1.png', '400', '156', 'with_mask', '191', '221', '67', '93'],
 ['maksssksksss1.png', '400', '156', 'with_mask', '21', '44', '73', '93'],
 ['maksssksksss1.png', '400', '156', 'with_mask', '369', '398', '70', '99'],
 ['maksssksksss1.png', '400', '156', 'without_mask', '83', '111', '56', '89'],
 ['maksssksksss10.png', '301', '400', '

In [9]:
df=pd.DataFrame(data,columns=['filename','width','height','name','xmin','xmax','ymin','ymax'])

In [10]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,maksssksksss0.png,512,366,without_mask,79,109,105,142
1,maksssksksss0.png,512,366,with_mask,185,226,100,144
2,maksssksksss0.png,512,366,without_mask,325,360,90,141
3,maksssksksss1.png,400,156,with_mask,321,354,34,69
4,maksssksksss1.png,400,156,with_mask,224,261,38,73


In [11]:
df.shape

(4072, 8)

In [12]:
df['name'].value_counts()

name
with_mask                3232
without_mask              717
mask_weared_incorrect     123
Name: count, dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4072 entries, 0 to 4071
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  4072 non-null   object
 1   width     4072 non-null   object
 2   height    4072 non-null   object
 3   name      4072 non-null   object
 4   xmin      4072 non-null   object
 5   xmax      4072 non-null   object
 6   ymin      4072 non-null   object
 7   ymax      4072 non-null   object
dtypes: object(8)
memory usage: 254.6+ KB


In [14]:
cols=['width','height','xmin','xmax','ymin','ymax']
df[cols]=df[cols].astype(int)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4072 entries, 0 to 4071
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  4072 non-null   object
 1   width     4072 non-null   int32 
 2   height    4072 non-null   int32 
 3   name      4072 non-null   object
 4   xmin      4072 non-null   int32 
 5   xmax      4072 non-null   int32 
 6   ymin      4072 non-null   int32 
 7   ymax      4072 non-null   int32 
dtypes: int32(6), object(2)
memory usage: 159.2+ KB


In [16]:
#centerx,centery
df['center_x']=((df['xmax']+df['xmin'])/2)/df['width']
df['center_y']=((df['ymax']+df['ymin'])/2)/df['height']

#w
df['w']=(df['xmax']-df['xmin'])/df['width']
#height
df['h']=(df['ymax']-df['ymin'])/df['height']

In [17]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,maksssksksss0.png,512,366,without_mask,79,109,105,142,0.183594,0.337432,0.058594,0.101093
1,maksssksksss0.png,512,366,with_mask,185,226,100,144,0.401367,0.333333,0.080078,0.120219
2,maksssksksss0.png,512,366,without_mask,325,360,90,141,0.668945,0.315574,0.068359,0.139344
3,maksssksksss1.png,400,156,with_mask,321,354,34,69,0.843750,0.330128,0.082500,0.224359
4,maksssksksss1.png,400,156,with_mask,224,261,38,73,0.606250,0.355769,0.092500,0.224359


In [18]:
#split the data into train and test
images=df['filename'].unique()
len(images)

853

In [19]:
img_df=pd.DataFrame(images,columns=['filename'])
img_train=tuple(img_df.sample(frac=0.8)['filename'])
img_test=tuple(img_df.query(f'filename not in{img_train}')['filename'])

In [21]:
train_df=df.query(f'filename in {img_train}')
test_df=df.query(f'filename in {img_test}')

In [22]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,maksssksksss0.png,512,366,without_mask,79,109,105,142,0.183594,0.337432,0.058594,0.101093
1,maksssksksss0.png,512,366,with_mask,185,226,100,144,0.401367,0.333333,0.080078,0.120219
2,maksssksksss0.png,512,366,without_mask,325,360,90,141,0.668945,0.315574,0.068359,0.139344
12,maksssksksss10.png,301,400,with_mask,98,194,267,383,0.485050,0.812500,0.318937,0.290000
13,maksssksksss100.png,400,226,with_mask,189,245,30,88,0.542500,0.261062,0.140000,0.256637


In [23]:
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
3,maksssksksss1.png,400,156,with_mask,321,354,34,69,0.84375,0.330128,0.0825,0.224359
4,maksssksksss1.png,400,156,with_mask,224,261,38,73,0.60625,0.355769,0.0925,0.224359
5,maksssksksss1.png,400,156,with_mask,299,315,58,81,0.76750,0.445513,0.0400,0.147436
6,maksssksksss1.png,400,156,with_mask,143,174,74,115,0.39625,0.605769,0.0775,0.262821
7,maksssksksss1.png,400,156,with_mask,74,95,69,99,0.21125,0.538462,0.0525,0.192308


In [24]:
#Assign id numbers to object names
def label_encoding(x):
    labels={'with_mask':0,'without_mask':1,'mask_weared_incorrect':2}
    return labels[x]

In [25]:
train_df['id']=train_df['name'].apply(label_encoding)
test_df['id']=test_df['name'].apply(label_encoding)

C:\Users\illas\AppData\Local\Temp\ipykernel_13748\1049210348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id']=train_df['name'].apply(label_encoding)
C:\Users\illas\AppData\Local\Temp\ipykernel_13748\1049210348.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id']=test_df['name'].apply(label_encoding)


In [26]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id
0,maksssksksss0.png,512,366,without_mask,79,109,105,142,0.183594,0.337432,0.058594,0.101093,1
1,maksssksksss0.png,512,366,with_mask,185,226,100,144,0.401367,0.333333,0.080078,0.120219,0
2,maksssksksss0.png,512,366,without_mask,325,360,90,141,0.668945,0.315574,0.068359,0.139344,1
12,maksssksksss10.png,301,400,with_mask,98,194,267,383,0.485050,0.812500,0.318937,0.290000,0
13,maksssksksss100.png,400,226,with_mask,189,245,30,88,0.542500,0.261062,0.140000,0.256637,0


In [34]:
import os
from shutil import move
train_folder='data_images/train'
test_folder='data_images/test'

os.mkdir(train_folder)
os.mkdir(test_folder)

cols=['filename','id','center_x','center_y','w','h']
groupby_obj_train=train_df[cols].groupby('filename')
groupby_obj_test=test_df[cols].groupby('filename')

In [35]:
groupby_obj_train.get_group('maksssksksss0.png').set_index('filename').to_csv('sample.txt',index=False,header=False)

In [40]:
print(groupby_obj_train)

In [37]:
filename_series =pd.Series(groupby_obj_train.groups.keys())

In [38]:
filename_series.apply(save_data,args=(train_folder,groupby_obj_train))

FileNotFoundError: [Errno 2] No such file or directory: 'data_images\\filename'

In [ ]:
with open('data.yaml.yaml',mode='r') as f:
           data_yaml=yaml.load(f,Loader=SafeLoader)

labels=data_yaml['names']


yolo=cv2.dnn.readNetFromONNX('./Model/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Load the image
img = cv2.imread('./prediction_img.png')

# Create a copy of the image
image = img.copy()

# Get the dimensions of the image
row, col, d = image.shape

# Now you can use the 'image' variable for further processing or analysis


#yolo pred from image
max_rc=max(row,col)
input_image=np.zeros((max_rc,max_rc,3),dtype=np.uint8)
input_image[0:row,0:col] = image

#get pred from square array
INPUT_WH_YOLO =640
blob=cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB=True,crop=False)
yolo.setInput(blob)
preds=yolo.forward()

detections=preds[0]
boxes=[]
confidences=[]
classes=[]
image_w,image_h=input_image.shape[:2]
x_factor =image_w/INPUT_WH_YOLO
y_factor =image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row=detections[i]
    confidence=row[4]
    if confidence > 0.4:
        class_score=row[5:].max()
        class_id = row[5:].argmax()
        
        if class_score >0.25:
            cx,cy,w,h=row[0:4]
            #construct bonding from four values
            #left,top,width,height
            left=int((cx-0.5*w)*x_factor)
            top=int((cy-0.5*h)*y_factor)
            width=int(w*x_factor)
            height=int(h*y_factor)
            #append the values into list
            box=np.array([left,top,width,height])
            
            confidences.append(confidences)
            boxes.append(box)
            classes.append(class_id)



#clean
boxes_np=np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

#NMS
index=cv2.dnn.NMSBoxes(boxes_np, confidences_np,0.5,0.5).flatten()

for ind in index:
    x,y,w,h = boxes_np[ind]
    bb_conf=int(confidences_np[ind]*100)
    classes_id=classes[ind]
    class_name=self.labels[classes_id]
    
    text=f'{class_name}: {bb_conf}%'
    
    cv2.rectangle(image,(x,y),(x+w,y),(0,255,0),2)
    cv2.rectangle(image,(x,y-30),(0,0,255),-1)
    cv2.putText(image,text,(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)

cv2.imshow('orginal',img)
cv2.imshow('yolo_prediction',image)
cv2.waitKey()